In [1]:
##Importação das bibliotecas necessárias
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import unicodedata


##Método para captura das informações dos jogadores, do TransferMarket
def get_player_info(club_url):

#Definição dos headers para request
    headers = {'User-Agent': 'Custom5'}
    response = requests.get(club_url, headers = headers)

#Utilização do Soup para parse do HTML obtido via request
    soup = bs(response.text,'html.parser')
    
    url_baseline = "https://www.transfermarkt.com"
    
    lista_jogadores = []
    lista_idades = []
    lista_posicoes = []
    lista_valores = []

#Utilização do soup para buscar objetos que retornem listas e que contenham cada uma das informações desejadas
    html_url = soup.find_all("div",{"class":"di nowrap"})
    jogadores = soup.find_all("img",{"class":"bilderrahmen-fixed lazy lazy"})
    idades = soup.find_all("td",{"class":"zentriert"})
    posicoes = soup.find_all("td", {"title":["Goalkeeper","Defender","midfield","attack"]})
    valores = soup.find_all("td", {"class":"rechts hauptlink"})

# obtenção do nome da equipe
    nome_da_equipe = str(soup.find_all("div", {"class":"subkategorie-header"})).split("</h2>",1)[0].split("Squad of ",1)[1].rstrip()

# obtenção das idaddes dos jogadores
    j = 1
    k = 1
    while j<= len(jogadores):
        try:
            lista_idades.append(str(idades[k]).split("(")[1].split(")")[0])
        except:
            lista_idades.append("N/A")
        j+=1
        k+=3


#obtenção do nome dos jogadores
    
    lista_semi_url = []
    for element in html_url:
        lista_semi_url.append(str(element).split('a href="')[1].split('" title=')[0])
    
    lista_semi_url = list(dict.fromkeys(lista_semi_url))
    
    for semi_url in lista_semi_url:
        full_player_url = url_baseline + semi_url
        player_response = requests.get(full_player_url, headers = headers)
        player_soup = bs(player_response.text,'html.parser')
        player_html_url = player_soup.find("span",{"class":"info-table__content info-table__content--bold"})
        player_full_name = str(player_html_url).split('bold">')[1].split('</span>')[0]
        lista_jogadores.append(unicodedata.normalize('NFD',player_full_name).encode('ascii', 'ignore').decode("utf-8"))

# obtenção das posições dos jogadores
    i=0
    for i in range(0,len(jogadores)):
        lista_posicoes.append(str(posicoes[i]).split('"><div class="',1)[0].split('title="',1)[1])

# obtenção dos valores dos jogadores
    for i in range(0,len(valores)):
        try:
            lista_valores.append((str(valores[i]).split('"><a')[1].split('">')[1].split("</a>")[0]))
        except:
            lista_valores.append("N/A")

# consolidando as info acima em um dataset
    df_final = pd.DataFrame({"Equipe": nome_da_equipe,
                             "Jogador": lista_jogadores,
                             "Posicao": lista_posicoes,
                             "Idade": lista_idades,
                             "Valor_de_Mercado" : lista_valores})
    return df_final, nome_da_equipe

In [2]:
#Geração do CSV com os dados do scrapper
def generate_csv(dataframe, local_path, nome_da_equipe, year):
    try:
        dataframe.to_csv(local_path + "/" + nome_da_equipe.replace(" ", "") + "_" + str(year) + ".csv" , index=False)
        print("Excel gerado com sucesso no caminho informado")
    except Exception as e:
        print("Houve um erro na geração do arquivo Excel --> " + e)
    return

In [3]:
#Criação de folder que irá armazenar os dados obtidos via scrapper
def create_folder(local_path, folder_name):
    full_path_name = local_path + "/" + folder_name
    flag_path_exists = os.path.exists(full_path_name)
    if not flag_path_exists:
        os.makedirs(full_path_name)
    return full_path_name